In [13]:
# !pip install python-arango nx-arangodb langchain_openai langchain_community langchain_mistralai mistralai langgraph

In [14]:
# !git clone https://github.com/pallets/flask.git

In [32]:
import os
import re
from random import randint
import nx_arangodb as nxadb
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from arango import ArangoClient
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_community.graphs import ArangoGraph
from langchain_community.chains.graph_qa.arangodb import ArangoGraphQAChain
from langchain_core.tools import tool
from mistralai import Mistral
from langchain_mistralai import ChatMistralAI

from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_extraction_chain

In [33]:
db = ArangoClient(hosts="https://d2eeb8083350.arangodb.cloud:8529").db(username="root", password="cUZ0YaNdcwfUTw6VjRny", verify=True)

print(db)

<StandardDatabase _system>


In [34]:
arango_graph = ArangoGraph(db)

In [35]:
G_adb = nxadb.Graph(
    name="FlaskRepv1",
    db=db,
    #incoming_graph_data=G,
    #write_batch_size=50000 # feel free to modify
)

print(G_adb)

[16:14:38 +0530] [INFO]: Graph 'FlaskRepv1' exists.
INFO:nx_arangodb:Graph 'FlaskRepv1' exists.
[16:14:39 +0530] [INFO]: Default node type set to 'FlaskRepv1_node'
INFO:nx_arangodb:Default node type set to 'FlaskRepv1_node'


Graph named 'FlaskRepv1' with 2975 nodes and 8652 edges


In [20]:
arango_graph = ArangoGraph(db)
arango_graph.schema

{'Graph Schema': [{'graph_name': 'CodebaseGraph',
   'edge_definitions': [{'edge_collection': 'CodebaseGraph_node_to_CodebaseGraph_node',
     'from_vertex_collections': ['CodebaseGraph_node'],
     'to_vertex_collections': ['CodebaseGraph_node']}]},
  {'graph_name': 'FlaskRepv1_node_to_FlaskRespv1_node',
   'edge_definitions': [{'edge_collection': 'FlaskRepv1_node_to_FlaskRespv1_node_node_to_FlaskRepv1_node_to_FlaskRespv1_node_node',
     'from_vertex_collections': ['FlaskRepv1_node_to_FlaskRespv1_node_node'],
     'to_vertex_collections': ['FlaskRepv1_node_to_FlaskRespv1_node_node']}]},
  {'graph_name': 'FlaskRepv1_node',
   'edge_definitions': [{'edge_collection': 'FlaskRepv1_node_node_to_FlaskRepv1_node_node',
     'from_vertex_collections': ['FlaskRepv1_node_node'],
     'to_vertex_collections': ['FlaskRepv1_node_node']}]},
  {'graph_name': 'FlaskRepv1',
   'edge_definitions': [{'edge_collection': 'FlaskRepv1_node_to_FlaskRepv1_node',
     'from_vertex_collections': ['FlaskRepv1_n

In [ ]:
def find_test_tag_interface_function(db, verbose=True):
    """
    Search for the test_tag_interface() function in the ArangoDB codebase.
    
    Args:
        db: The ArangoDB database connection object
        verbose: Whether to print results as they're found
    
    Returns:
        A list of dictionaries containing the search results
    """
    # Query to search for the test_tag_interface() function
    query = """
    // Try main collection first
    LET main_results = (
        FOR doc IN FlaskRepv1_node
            FILTER CONTAINS(doc.content, "test_tag_interface()")
            LET pos = POSITION(doc.content, "test_tag_interface()")
            LET start_pos = pos > 100 ? pos - 100 : 0
            LET content_preview = SUBSTRING(doc.content, start_pos, 300)
            RETURN {
                collection: "FlaskRepv1_node",
                document_id: doc._id,
                file_path: doc.directory,
                file_index: doc.file_index,
                content_preview: content_preview
            }
    )
    
    // Try other collections if no results found
    LET other_results = (
        LENGTH(main_results) == 0 ? (
            FOR collection IN ["CodebaseGraph_node", "FlaskRepv1_node_to_FlaskRespv1_node_node", "FlaskRepv1_node_node"]
                FOR doc IN @@collection
                    FILTER IS_DOCUMENT(doc) AND HAS(doc, "content") AND CONTAINS(doc.content, "test_tag_interface()")
                    LET pos = POSITION(doc.content, "test_tag_interface()")
                    LET start_pos = pos > 100 ? pos - 100 : 0
                    LET content_preview = SUBSTRING(doc.content, start_pos, 300)
                    RETURN {
                        collection: collection,
                        document_id: doc._id,
                        file_path: HAS(doc, "directory") ? doc.directory : null,
                        file_index: HAS(doc, "file_index") ? doc.file_index : null,
                        content_preview: content_preview
                    }
        ) : []
    )
    
    // Combine results
    LET all_results = APPEND(main_results, other_results)
    
    // If function found, check for related nodes in graph
    LET graph_results = (
        LENGTH(all_results) > 0 ? (
            FOR hit IN all_results
                LET doc = DOCUMENT(hit.document_id)
                LET connected = (
                    FOR v, e IN 1..2 OUTBOUND doc FlaskRepv1_node_to_FlaskRepv1_node
                    RETURN {
                        related_id: v._id,
                        relationship: e.edge_type,
                        related_type: HAS(v, "type") ? v.type : null,
                        preview: HAS(v, "content") ? SUBSTRING(v.content, 0, 150) : null
                    }
                )
                RETURN MERGE(hit, {related_nodes: connected})
        ) : all_results
    )
    
    RETURN graph_results
    """
    
    try:
        # Execute the query
        cursor = db.aql.execute(query, bind_vars={"@collection": "FlaskRepv1_node"})
        results = [doc for doc in cursor]
        
        # Flatten results (the query returns a list of lists)
        if results and isinstance(results[0], list):
            results = results[0]
        
        # Print results if verbose
        if verbose:
            print(f"Found {len(results)} occurrences of test_tag_interface() function:")
            for i, result in enumerate(results, 1):
                print(f"\n--- Result {i} ---")
                print(f"Collection: {result.get('collection')}")
                print(f"Document ID: {result.get('document_id')}")
                print(f"File Path: {result.get('file_path')}")
                print(f"File Index: {result.get('file_index')}")
                print(f"\nContent Preview:")
                print(f"```\n{result.get('content_preview')}\n```")
                
                # Print related nodes if available
                if 'related_nodes' in result and result['related_nodes']:
                    print(f"\nRelated Nodes ({len(result['related_nodes'])}):")
                    for j, node in enumerate(result['related_nodes'][:3], 1):  # Show only first 3 related nodes
                        print(f"  {j}. {node.get('related_id')} ({node.get('relationship')})")
                    
                    if len(result['related_nodes']) > 3:
                        print(f"  ... and {len(result['related_nodes']) - 3} more related nodes")
        
        return results
            
    except Exception as e:
        print(f"Error executing ArangoDB query: {str(e)}")
        return []

# Example usage:
# from arango import ArangoClient
# 
# # Connect to ArangoDB
# client = ArangoClient(hosts="http://localhost:8529")
# db = client.db("_system", username="root", password="your_password")
# 
# # Find the test_tag_interface() function
find_test_tag_interface_function(db)

Found 0 occurrences of test_tag_interface() function:


[]

In [21]:
os.environ["MISTRAL_API_KEY"]="jJAuJZkjVcy2ynUhan375sHNviHiBeJU"

In [22]:
llm = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0,
    max_retries=2,
    api_key = os.environ["MISTRAL_API_KEY"]
)


In [29]:
import os
import re
from random import randint
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nx_arangodb as nxadb

from arango import ArangoClient
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_community.graphs import ArangoGraph
from langchain_community.chains.graph_qa.arangodb import ArangoGraphQAChain
from langchain_core.tools import tool
from mistralai import Mistral
from langchain_mistralai import ChatMistralAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_extraction_chain
from langchain_core.messages import HumanMessage, AIMessage

# # Connect to ArangoDB
# db = ArangoClient(hosts="https://d2eeb8083350.arangodb.cloud:8529").db(
#     username="root", password="cUZ0YaNdcwfUTw6VjRny", verify=True
# )
# arango_graph = ArangoGraph(db)

# # Connect to NetworkX representation
# G_adb = nxadb.Graph(
#     name="FlaskRepv1",
#     db=db
# )

# # Initialize MistralAI LLM
# llm = ChatMistralAI(
#     model="mistral-large-latest",
#     temperature=0,
#     max_retries=2,
# )

# Tool for querying ArangoDB using natural language
@tool
def text_to_aql_to_text(query: str):
    """Query the repository graph database using natural language.
    This tool translates your query to AQL, executes it, and returns results.
    Use for finding files, content, relationships between files, etc.
    """
    chain = ArangoGraphQAChain.from_llm(
        llm=llm,
        graph=arango_graph,
        verbose=True,
        allow_dangerous_requests=True
    )
    result = chain.invoke(query)
    return str(result["result"])

# Tool for using NetworkX algorithms on the graph
@tool
def text_to_nx_algorithm_to_text(query: str):
    """Apply graph algorithms to analyze the repository structure.
    This tool is ideal for complex queries like finding dependencies,
    centrality of files, or traversal patterns.
    """
    # Generate NetworkX code based on query
    text_to_nx = llm.invoke(f"""
    I have a NetworkX Graph called `G_adb`. It has the following schema: {arango_graph.schema}

    I have the following graph analysis query: {query}.

    Generate the Python Code required to answer the query using the `G_adb` object.

    Be very precise on the NetworkX algorithm you select to answer this query. Think step by step.

    Only assume that networkx is installed, and other base python dependencies.

    Always set the last variable as `FINAL_RESULT`, which represents the answer to the original query.

    Only provide python code that I can directly execute via `exec()`. Do not provide any instructions.

    Make sure that `FINAL_RESULT` stores a short & concise answer. Avoid setting this variable to a long sequence.

    Your code:
    """).content

    text_to_nx_cleaned = re.sub(r"^```python\n|```$", "", text_to_nx, flags=re.MULTILINE).strip()

    # Execute the generated code
    global_vars = {"G_adb": G_adb, "nx": nx}
    local_vars = {}

    try:
        exec(text_to_nx_cleaned, global_vars, local_vars)
        FINAL_RESULT = local_vars.get("FINAL_RESULT", "No result was produced")
    except Exception as e:
        return f"Error executing graph algorithm: {str(e)}"

    # Format the response
    nx_to_text = llm.invoke(f"""
        I have a NetworkX Graph called `G_adb`. It has the following schema: {arango_graph.schema}

        I have the following graph analysis query: {query}.

        I have executed NetworkX algorithms and the result is: {FINAL_RESULT}

        Based on this result, provide a clear and helpful answer to the original query.

        Your response:
    """).content

    return nx_to_text

# Tool to search file contents for error messages or specific patterns
@tool
def search_code_for_pattern(pattern: str):
    """Search through repository files for specific code patterns, error messages,
    or function definitions. Returns relevant file sections that match the pattern.
    """
    try:
        # Use AQL to find files that might contain the pattern
        query = f"""
        FOR file IN Files
            FILTER LIKE(file.content, '%{pattern}%', true)
            RETURN {{
                filename: file.name,
                path: file.path,
                matching_content: SUBSTRING(file.content,
                                          MAX(0, POSITION(file.content, '{pattern}', true) - 100),
                                          MIN(200, LENGTH(file.content) - MAX(0, POSITION(file.content, '{pattern}', true) - 100)))
            }}
        """
        cursor = db.aql.execute(query)
        results = list(cursor)

        if not results:
            return f"No files found containing the pattern '{pattern}'."

        response = f"Found {len(results)} file(s) containing '{pattern}':\n\n"
        for result in results:
            response += f"File: {result['path']}/{result['filename']}\n"
            response += f"Relevant section: ```\n{result['matching_content']}\n```\n\n"

        return response
    except Exception as e:
        return f"Error searching for pattern: {str(e)}"

# Tool to find dependencies between files
@tool
def find_dependencies(file_path: str):
    """Find which files import or are imported by a specific file.
    Useful for understanding dependencies and potential impact of changes.
    """
    try:
        # Find files that this file imports
        imports_query = f"""
        FOR v, e IN 1..1 OUTBOUND 'Files/{file_path}' Imports
            RETURN {{
                imported_file: v.name,
                imported_path: v.path,
                relationship: 'imported by {file_path}'
            }}
        """
        imports_cursor = db.aql.execute(imports_query)
        imports = list(imports_cursor)

        # Find files that import this file
        imported_by_query = f"""
        FOR v, e IN 1..1 INBOUND 'Files/{file_path}' Imports
            RETURN {{
                importing_file: v.name,
                importing_path: v.path,
                relationship: 'imports {file_path}'
            }}
        """
        imported_by_cursor = db.aql.execute(imported_by_query)
        imported_by = list(imported_by_cursor)

        response = f"Dependency analysis for {file_path}:\n\n"

        if imports:
            response += "Files imported by this file:\n"
            for imp in imports:
                response += f"- {imp['imported_path']}/{imp['imported_file']}\n"
        else:
            response += "This file does not import any other files in the repository.\n"

        response += "\n"

        if imported_by:
            response += "Files that import this file:\n"
            for imp in imported_by:
                response += f"- {imp['importing_path']}/{imp['importing_file']}\n"
        else:
            response += "This file is not imported by any other files in the repository.\n"

        return response
    except Exception as e:
        return f"Error finding dependencies: {str(e)}"

# NEW TOOL: Codebase Explainer using graph visualization and analysis
@tool
def explain_codebase_structure(topic_or_feature: str):
    """Explain the codebase structure related to a specific topic or feature.
    This tool combines graph analysis, file content search, and dependency mapping
    to provide a comprehensive explanation of how the code is organized.
    """
    try:
        # Step 1: Find relevant files
        relevant_files_query = f"""
        FOR file IN Files
            FILTER LIKE(file.content, '%{topic_or_feature}%', true)
            RETURN {{
                id: file._id,
                name: file.name,
                path: file.path,
                type: file.type,
                size: file.size
            }}
        LIMIT 10
        """
        relevant_cursor = db.aql.execute(relevant_files_query)
        relevant_files = list(relevant_cursor)

        if not relevant_files:
            # Try broader search if specific search yields no results
            broader_query = f"""
            FOR file IN Files
                SORT file.size DESC
                RETURN {{
                    id: file._id,
                    name: file.name,
                    path: file.path,
                    type: file.type,
                    size: file.size
                }}
            LIMIT 5
            """
            broader_cursor = db.aql.execute(broader_query)
            relevant_files = list(broader_cursor)

        # Step 2: Find relationships between these files
        file_relationships = []
        for file in relevant_files:
            file_id = file['id']

            # Find incoming relationships
            in_rel_query = f"""
            FOR v, e IN 1..1 INBOUND '{file_id}' Imports
                RETURN {{
                    source: v.name,
                    source_path: v.path,
                    target: '{file['name']}',
                    relationship: 'imports'
                }}
            """
            in_cursor = db.aql.execute(in_rel_query)
            file_relationships.extend(list(in_cursor))

            # Find outgoing relationships
            out_rel_query = f"""
            FOR v, e IN 1..1 OUTBOUND '{file_id}' Imports
                RETURN {{
                    source: '{file['name']}',
                    source_path: '{file['path']}',
                    target: v.name,
                    relationship: 'imports'
                }}
            """
            out_cursor = db.aql.execute(out_rel_query)
            file_relationships.extend(list(out_cursor))

        # Step 3: Get code snippets from key files
        code_snippets = []
        for file in relevant_files[:3]:  # Limit to top 3 files
            file_content_query = f"""
            FOR file IN Files
                FILTER file._id == '{file['id']}'
                RETURN file.content
            """
            content_cursor = db.aql.execute(file_content_query)
            content_list = list(content_cursor)

            if content_list:
                content = content_list[0]
                # Extract relevant section containing the topic
                if topic_or_feature in content:
                    start_pos = max(0, content.find(topic_or_feature) - 200)
                    end_pos = min(len(content), content.find(topic_or_feature) + 200)
                    relevant_content = content[start_pos:end_pos]
                    code_snippets.append({
                        "file": f"{file['path']}/{file['name']}",
                        "snippet": relevant_content
                    })

        # Step 4: Generate a comprehensive explanation using the LLM
        explanation_prompt = f"""
        I need to explain the structure of a codebase related to "{topic_or_feature}".

        Here are the relevant files:
        {relevant_files}

        Here are the relationships between these files:
        {file_relationships}

        Here are code snippets from key files:
        {code_snippets}

        Based on this information, provide a clear explanation of:
        1. The overall architecture related to {topic_or_feature}
        2. How the files interact with each other
        3. Key components and their responsibilities
        4. How a developer should navigate this part of the codebase

        Your explanation should be comprehensive but easy to understand.
        """

        explanation = llm.invoke(explanation_prompt).content

        return explanation
    except Exception as e:
        return f"Error explaining codebase structure: {str(e)}"

# Create a system prompt for the RAG agent
system_template = """You are a helpful coding assistant with access to a graph database containing information about a repository's files, folders, and their contents.

When a user presents an error or asks for help with a feature, you will:
1. Analyze their query to understand what they need
2. Use the graph database tools to retrieve relevant information from the repository
3. Provide clear solutions or explanations based on the actual code in the repository

Repository Schema:
{schema}

Available tools:
- text_to_aql_to_text: Query the graph database in natural language
- text_to_nx_algorithm_to_text: Apply graph algorithms to analyze repository structure
- search_code_for_pattern: Search code for specific patterns or error messages
- find_dependencies: Find relationships between files
- explain_codebase_structure: Get a comprehensive explanation of the codebase structure for specific topics or features

Answer the user's questions based on the actual repository structure and code.
"""

# Create the RAG system
tools = [text_to_aql_to_text, text_to_nx_algorithm_to_text, search_code_for_pattern, find_dependencies, explain_codebase_structure]

# Updated create_rag_agent function to fix the parameter issue
def create_rag_agent():
    """Create a RAG agent that can query the repository graph database"""
    # Create a template with the schema already processed
    formatted_system_message = system_template.format(schema=arango_graph.schema)

    # Create the agent directly with the formatted message
    agent = create_react_agent(
        model=llm,
        tools=tools,
        prompt=formatted_system_message  # Try using system_message directly
    )

    return agent
# Function to handle user queries
def handle_repository_query(user_query):
    """Process user queries about the repository"""
    agent = create_rag_agent()
    result = agent.invoke({
        "messages": [
            {"role": "user", "content": user_query}
        ]
    })
    return result["messages"][-1].content

# Example usage
if __name__ == "__main__":
    # Sample queries to test the system
    test_queries = [
        "Explain me the test_tag_interface() function in the codebase?",
    ]

    print("Repository RAG System Demo")
    print("=" * 50)

    for i, query in enumerate(test_queries, 1):
        print(f"\nTest Query {i}: {query}")
        print("-" * 50)
        response = handle_repository_query(query)
        print(f"Response: {response}")
        print("=" * 50)

Repository RAG System Demo

Test Query 1: Explain me the test_tag_interface() function in the codebase?
--------------------------------------------------


> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH FlaskRepv1_node, FlaskRepv1_node_to_FlaskRepv1_node
FOR vertex IN FlaskRepv1_node
  FILTER vertex.type == 'function' && vertex.name == 'test_tag_interface'
  RETURN vertex

AQL Result:
[]

> Finished chain.


HTTPStatusError: Error response 429 while fetching https://api.mistral.ai/v1/chat/completions: {"message":"Requests rate limit exceeded"}

In [26]:
text_to_nx_algorithm_to_text("Explain me the test_tag_interface() function in the codebase.?")

1) Generating NetworkX code
----------
import networkx as nx

# Assuming G_adb is already defined and loaded with the graph data

# Step 1: Identify the node corresponding to the function test_tag_interface()
function_name = 'test_tag_interface'
function_node = None
for node, data in G_adb.nodes(data=True):
    if data.get('type') == 'function' and data.get('name') == function_name:
        function_node = node
        break

# Step 2: Use BFS to explore the neighborhood of the function node
if function_node:
    bfs_tree = nx.bfs_tree(G_adb, source=function_node)
    bfs_nodes = list(bfs_tree.nodes)

    # Step 3: Collect information about the nodes in the BFS tree
    function_info = []
    for node in bfs_nodes:
        node_data = G_adb.nodes[node]
        function_info.append(node_data)

    # Step 4: Summarize the information
    summary = f"The function {function_name} is connected to the following nodes: "
    for info in function_info:
        summary += f"{info.get('name')} (

'The function `test_tag_interface` was not found in the graph. Therefore, I cannot provide an explanation for it based on the current graph data.'

In [25]:
def text_to_nx_algorithm_to_text(query):
    """This tool is available to invoke a NetworkX Algorithm on
    the ArangoDB Graph. You are responsible for accepting the
    Natural Language Query, establishing which algorithm needs to
    be executed, executing the algorithm, and translating the results back
    to Natural Language, with respect to the original query.

    If the query (e.g traversals, shortest path, etc.) can be solved using the Arango Query Language, then do not use
    this tool.
    """


    ######################
    print("1) Generating NetworkX code")

    text_to_nx = llm.invoke(f"""
    I have a NetworkX Graph called `G_adb`. It has the following schema: {arango_graph.schema}

    I have the following graph analysis query: {query}.

    It is an undirected graph.
    
    Generate the Python Code required to answer the query using the `G_adb` object.

    Be very precise on the NetworkX algorithm you select to answer this query. Think step by step.

    Only assume that networkx is installed, and other base python dependencies.

    Always set the last variable as `FINAL_RESULT`, which represents the answer to the original query.

    Only provide python code that I can directly execute via `exec()`. Do not provide any instructions.

    Make sure that `FINAL_RESULT` stores a short & consice answer. Avoid setting this variable to a long sequence.

    Your code:
    """).content

    text_to_nx_cleaned = re.sub(r"^```python\n|```$", "", text_to_nx, flags=re.MULTILINE).strip()

    print('-'*10)
    print(text_to_nx_cleaned)
    print('-'*10)

    ######################

    print("\n2) Executing NetworkX code")
    global_vars = {"G_adb": G_adb, "nx": nx}
    local_vars = {}

    try:
        exec(text_to_nx_cleaned, global_vars, local_vars)
        text_to_nx_final = text_to_nx
    except Exception as e:
        print(f"EXEC ERROR: {e}")
        return f"EXEC ERROR: {e}"

        # TODO: Consider experimenting with a code corrector!
        attempt = 1
        MAX_ATTEMPTS = 3

        # while attempt <= MAX_ATTEMPTS
            # ...

    print('-'*10)
    FINAL_RESULT = local_vars["FINAL_RESULT"]
    print(f"FINAL_RESULT: {FINAL_RESULT}")
    print('-'*10)

    ######################

    print("3) Formulating final answer")

    nx_to_text = llm.invoke(f"""
        I have a NetworkX Graph called `G_adb`. It has the following schema: {arango_graph.schema}

        I have the following graph analysis query: {query}.

        I have executed the following python code to help me answer my query:

        ---
        {text_to_nx_final}
        ---

        The `FINAL_RESULT` variable is set to the following: {FINAL_RESULT}.

        Based on my original Query and FINAL_RESULT, generate a short and concise response to
        answer my query.

        Your response:
    """).content

    return nx_to_text

In [30]:

# Tool for using NetworkX algorithms on the graph
@tool
def text_to_nx_algorithm_to_text(query: str):
    """Apply graph algorithms to analyze the repository structure.
    This tool is ideal for complex queries like finding dependencies,
    centrality of files, or traversal patterns.
    """
    # Generate NetworkX code based on query
    text_to_nx = llm.invoke(f"""
    I have a NetworkX Graph called `G_adb`. It has the following schema: {arango_graph.schema}

    I have the following graph analysis query: {query}.

    Generate the Python Code required to answer the query using the `G_adb` object.

    Be very precise on the NetworkX algorithm you select to answer this query. Think step by step.

    Only assume that networkx is installed, and other base python dependencies.

    Always set the last variable as `FINAL_RESULT`, which represents the answer to the original query.

    Only provide python code that I can directly execute via `exec()`. Do not provide any instructions.

    Make sure that `FINAL_RESULT` stores a short & concise answer. Avoid setting this variable to a long sequence.

    Your code:
    """).content

    text_to_nx_cleaned = re.sub(r"^```python\n|```$", "", text_to_nx, flags=re.MULTILINE).strip()

    # Execute the generated code
    global_vars = {"G_adb": G_adb, "nx": nx}
    local_vars = {}

    try:
        exec(text_to_nx_cleaned, global_vars, local_vars)
        FINAL_RESULT = local_vars.get("FINAL_RESULT", "No result was produced")
    except Exception as e:
        return f"Error executing graph algorithm: {str(e)}"

    # Format the response
    nx_to_text = llm.invoke(f"""
        I have a NetworkX Graph called `G_adb`. It has the following schema: {arango_graph.schema}

        I have the following graph analysis query: {query}.

        I have executed NetworkX algorithms and the result is: {FINAL_RESULT}

        Based on this result, provide a clear and helpful answer to the original query.

        Your response:
    """).content

    return nx_to_text

# Example usage
if __name__ == "__main__":
    # Sample query to test the system
    query = "Explain me the test_tag_interface() function in the codebase."
    response = text_to_nx_algorithm_to_text(query)
    print(response)

Based on the analysis of the NetworkX Graph `G_adb` and the schema provided, it appears that the function `test_tag_interface` was not found in the codebase. This means that there is no node or edge in the graph that corresponds to this function.

Here are a few possible reasons for this result:

1. **Function Not Present**: The function `test_tag_interface` might not exist in the codebase. It could be that the function was never implemented, or it might be part of a different codebase or version that is not included in the current graph.

2. **Naming Convention**: There might be a discrepancy in the naming convention. The function could be named differently in the codebase, or there could be a typo in the query.

3. **Graph Incompleteness**: The graph might not be fully populated with all the nodes and edges representing the codebase. This could happen if the graph construction process missed some parts of the codebase.

4. **Scope Limitation**: The function might be part of a differe